In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/eff-btrfly

/content/drive/MyDrive/Colab Notebooks/eff-btrfly


In [4]:
!pip install --upgrade efficientnet-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=6c5cd665975d1b66658836796fbff9057638ef685a46dee5bb14a395e3cbbc5e
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [5]:
#arsitektur Efficient-BtrflyNet 
#from btreff import BtrflyNet
from varianbtreff import BtrflyNet #coba varian EfficientNet

In [6]:
def posterior_prs(pos):
    tmp = pos.argmax(axis=1)
    for i in range(7, 13):
        tmp = tmp.where(tmp != i, torch.tensor(i-1))
    return tmp

In [7]:
import torch
import torch.nn.functional as func

def compute_loss(gt_ant, gt_pos, out_ant, out_pos):

    loss_ant = func.cross_entropy(out_ant, gt_ant.argmax(axis=1))
    loss_pos = func.cross_entropy(out_pos, gt_pos.argmax(axis=1))

    return loss_ant + loss_pos

In [8]:
import matplotlib.pyplot as plt
import numpy as np

imsize = (512, 128, 1)

def masking(msk):
    tmp = msk.round()
    return [(tmp == i).astype('float').tolist() for i in range(13)]

In [9]:
from PIL import Image
from torchvision import transforms

tfms = transforms.Compose([transforms.ToTensor()])

In [10]:
import os

X, y = [], []

for name in os.listdir(f'./data/img'):
    if name[-5] == '1':
        X.append([
            tfms(Image.open(f'./data/img/{name[:-5]}1.png')).tolist(),
            tfms(Image.open(f'./data/img/{name[:-5]}2.png')).tolist()
        ])
        y.append([
            masking(plt.imread(f'./data/msk/{name[:-5]}1.png')[...,0] * 12),
            masking(plt.imread(f'./data/msk/{name[:-5]}2.png')[...,0] * 12)
        ])

vl = [ 6, 29,  0, 22, 34, 30,  4, 31]
tr = [i for i in range(37) if i not in vl]

X_train, X_valid = torch.Tensor( [X[i] for i in tr] ), torch.Tensor( [X[j] for j in vl] )
y_train, y_valid = torch.Tensor( [y[i] for i in tr] ), torch.Tensor( [y[j] for j in vl] )

n_data_train, n_data_valid = len(X_train), len(X_valid)

In [11]:
device     = torch.device('cuda')
best_valid = np.inf

In [12]:
import pickle

model = BtrflyNet().to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/adv-efficientnet-b7-4652b6dd.pth" to /root/.cache/torch/hub/checkpoints/adv-efficientnet-b7-4652b6dd.pth
100%|██████████| 255M/255M [00:03<00:00, 76.7MB/s]


Loaded pretrained weights for efficientnet-b7
Loaded pretrained weights for efficientnet-b7


In [13]:
def dice_coef(y_true, y_pred):
    y_true_f = y_true[:,:,1:,:,:].flatten()
    y_pred_f = y_pred[:,:,1:,:,:].flatten()
    union = y_true_f.flatten().sum().item() + y_pred_f.flatten().sum().item()
    if union == 0: return 1
    intersection = (y_true_f * y_pred_f).sum().item()
    return 2 * intersection / union

In [14]:
def masking_torch(msk):
    return torch.Tensor([ (msk == i).cpu().numpy().astype('float') for i in range(13) ])

In [ ]:
import torchvision.transforms as trans
import timeit

from tqdm.notebook import tqdm

epochs   = 800 
inp_size = (1,  3, 512, 128)
out_size = (1, 13, 512, 128)
dsc_size = (1, 1, 13, 512, 128)

#coba ubah optimizer dan lr
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

train_losses = []
valid_losses = []

start = timeit.default_timer()

for epoch in tqdm( range(epochs) ):
    
    model.train()

    total_loss_train = 0
    total_loss_valid = 0
    
    for i in range(n_data_train):

        model.zero_grad()
        
        x, y = X_train[i], y_train[i]

        if np.random.random() > 0.5:
            h_flip = trans.RandomHorizontalFlip(1)
            x, y = h_flip(x), h_flip(y)
        
        out_ant, out_pos = model(
            x[0].reshape(inp_size).to(device),
            x[1].reshape(inp_size).to(device)
        )

        loss_train = compute_loss(
            y[0].reshape(out_size).to(device),
            y[1].reshape(out_size).to(device),
            out_ant, out_pos
        )
        
        total_loss_train += loss_train.item()
        
        loss_train.backward()

        optimizer.step()
        optimizer.zero_grad()

    with torch.no_grad():
    
        for i in range(n_data_valid):
            
            x, y = X_valid[i], y_valid[i]
            
            out_ant, out_pos = model(
                x[0].reshape(inp_size).to(device),
                x[1].reshape(inp_size).to(device)
            )

            tmp = torch.cat([
                masking_torch(out_ant.argmax(axis=1)[0]).reshape(dsc_size),
                masking_torch(out_pos.argmax(axis=1)[0]).reshape(dsc_size)
            ], axis=1)

            if i == 0: y_predv = tmp + 0
            else: y_predv = torch.cat([y_predv, tmp], axis=0)

            loss_valid = compute_loss(
                y[0].reshape(out_size).to(device),
                y[1].reshape(out_size).to(device),
                out_ant, out_pos
            )

            total_loss_valid += loss_valid.item()

        for i in range(n_data_train):
            
            x, y = X_train[i], y_train[i]
            
            out_ant, out_pos = model(
                x[0].reshape(inp_size).to(device),
                x[1].reshape(inp_size).to(device)
            )

            tmp = torch.cat([
                masking_torch(out_ant.argmax(axis=1)[0]).reshape(dsc_size),
                masking_torch(out_pos.argmax(axis=1)[0]).reshape(dsc_size)
            ], axis=1)

            if i == 0: y_predt = tmp + 0
            else: y_predt = torch.cat([y_predt, tmp], axis=0)

    train_losses.append(total_loss_train / n_data_train)
    valid_losses.append(total_loss_valid / n_data_valid)

    scheduler.step(valid_losses[epoch])
    
    #print(f'')
    #print(f'- tr loss = {train_losses[epoch]:.4f}')
    #print(f'- vl loss = {valid_losses[epoch]:.4f}')
    #print(f'- tr dice = {dice_coef(y_train, y_predt)}')
    #print(f'- vl dice = {dice_coef(y_valid, y_predv)}')

    if best_valid > valid_losses[epoch]:
        best_valid = valid_losses[epoch] + 0
        print(f'')
        print(f'- tr loss = {train_losses[epoch]:.4f}')
        print(f'- vl loss = {valid_losses[epoch]:.4f}')
        print(f'- tr dice = {dice_coef(y_train, y_predt)}')
        print(f'- vl dice = {dice_coef(y_valid, y_predv)}')
        torch.save(model.state_dict(), f'./kupu-model/model-eff0406b.pt')
        print('model saved')

stop = timeit.default_timer()

print('Time: ', stop - start)

  0%|          | 0/800 [00:00<?, ?it/s]

<ipython-input-14-89ed43dc3f24>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  return torch.Tensor([ (msk == i).cpu().numpy().astype('float') for i in range(13) ])



- tr loss = 5.0791
- vl loss = 5.0270
- tr dice = 0.07582829771494798
- vl dice = 0.07718829283293809
model saved

- tr loss = 4.9831
- vl loss = 4.9410
- tr dice = 0.1340340710378843
- vl dice = 0.13621388926746966
model saved

- tr loss = 4.9123
- vl loss = 4.8814
- tr dice = 0.16620742151941684
- vl dice = 0.168240464500988
model saved

- tr loss = 4.8609
- vl loss = 4.8385
- tr dice = 0.1892797916410602
- vl dice = 0.19525408167780187
model saved

- tr loss = 4.8219
- vl loss = 4.8021
- tr dice = 0.21240714658094917
- vl dice = 0.21710324829134214
model saved

- tr loss = 4.7881
- vl loss = 4.7712
- tr dice = 0.24896912162174395
- vl dice = 0.254147520089245
model saved

- tr loss = 4.7601
- vl loss = 4.7462
- tr dice = 0.27947081002729385
- vl dice = 0.2862653290491865
model saved

- tr loss = 4.7375
- vl loss = 4.7253
- tr dice = 0.3008673677656947
- vl dice = 0.3066065054176714
model saved

- tr loss = 4.7159
- vl loss = 4.7022
- tr dice = 0.327993688888686
- vl dice = 0.333282

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')

plt.legend()

In [ ]:
full_loss = (train_losses, valid_losses)
with open(f'./kupu-model/loss-eff0406b.pickle', 'wb') as f: pickle.dump(full_loss, f)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-6e2531f94912>", line 2, in <cell line: 2>
    with open(f'./kupu-model/loss-eff0406b.pickle', 'wb') as f: pickle.dump(full_loss, f)
OSError: [Errno 107] Transport endpoint is not connected: './kupu-model/loss-eff0406b.pickle'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinn